In [ ]:
raster_patj = ""
stats = rasterstats.zonal_stats(raster.geometry(), raster_paths, stats=['mean', 'median', 'min', 'max', 'sum'])


In [1]:
raster_path = "/soc_l8_Bolangir_23_30m_masked_tons.tif"

In [2]:
import rasterio
import rasterstats
from shapely.geometry import box

# Open the raster and get its bounds
with rasterio.open(raster_path) as src:
    bounds = src.bounds

# Create a geometry that covers the whole raster
geometry = [box(*bounds)]

# Compute the statistics
stats = rasterstats.zonal_stats(geometry, raster_path, stats=['mean', 'median', 'min', 'max', 'sum'])

print(stats)

/Users/najah/miniforge3/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


[{'min': 8.864177325367928, 'max': 15.151887479424476, 'mean': 12.628264878396545, 'median': 12.670277559757233}]


In [3]:
import rasterio
import rasterstats
from shapely.geometry import box

import pandas as pd
datasets = {22: ['Bolangir', 'Medak', 'Yavatmal'], 23: ['Bolangir', 'Medak', 'Yavatmal']}
results = []
for year, regions in datasets.items():
        for region in regions:
            print(region)
            raster_path = f'/new_out/soc_l8_{region}_{year}_30m_masked_esri_tons.tif'
        # Open the raster and get its bounds
            with rasterio.open(raster_path) as src:
                bounds = src.bounds

            # Create a geometry that covers the whole raster
            geometry = [box(*bounds)]

            # Compute the statistics
            stats = rasterstats.zonal_stats(geometry, raster_path, stats=['mean', 'median', 'min', 'max', 'sum'])

            #print(stats)
            # add region and year

            stats[0]['region']= region
            stats[0]['year']= year

            # append the results

            results.append(stats[0])

df = pd.DataFrame(results)    
# Write the DataFrame to a CSV file
df.to_csv('zonal_stats_new.csv', index=False)        

Bolangir
Medak
Yavatmal
Bolangir
Medak
Yavatmal


In [4]:
df

,min,max,mean,sum,median,region,year
0,6.304927,9.983869,8.336409,2.833978e+07,8.329373,Bolangir,22
1,5.682813,10.412733,7.865303,6.287914e+07,7.848365,Medak,22
2,5.662558,9.851213,7.694682,7.423342e+07,7.596926,Yavatmal,22
3,6.219724,9.824923,8.342904,2.836186e+07,8.341067,Bolangir,23
4,5.534774,10.532301,7.846703,6.273044e+07,7.810954,Medak,23
5,5.654472,9.527905,7.635696,7.366436e+07,7.527410,Yavatmal,23


In [15]:
type(stats[0])

dict

In [20]:
import pandas as pd
pd.DataFrame(stats)

,min,max,mean,median
0,8.542728,14.640439,11.595337,11.464376


## subdistrict wise

In [26]:
import overpy

# Initialize the Overpass API client
api = overpy.Overpass()

# Specify the name of the district
district_name = 'Bolangir'

# Query the Overpass API for the subdistricts within the district
result = api.query(f"""
    area["name"="{district_name}"]["admin_level"="6"];
    (relation(area)["admin_level"="7"];
    way(area)["admin_level"="7"];
    node(area)["admin_level"="7"];);
    out;
""")

# Print the names of the subdistricts
for relation in result.relations:
    print(relation.tags.get('name', 'Unnamed'))